<a href="https://colab.research.google.com/github/raquelcarmo/tropical_cyclones/blob/main/src/code/TC_Vmax_Regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Maximum Wind Speed (Vmax) Regression
Script to execute training of a regression model using the Maximum Wind Speed feature of the Tropical Cyclone.

## Imports and configurations

In [ ]:
# Insert your desired path to work on
import os
os.chdir('../data')

Install requirements.

In [ ]:
# General imports
import pandas as pd
import numpy as np
import cv2
import matplotlib.pyplot as plt
import time
import netCDF4
from glob import glob
from datetime import datetime

import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

np.set_printoptions(precision=4)

Load functions to be used.

In [ ]:
def knots_to_m_sec(kts):
    """Converts knots (kt) to meters per second (m/s)."""
    if np.isnan(kts):
        return kts
    else:
        return kts*0.514444

Set folder structure.

In [ ]:
config = {
    'in_nc': 'nc',
    'out_vmax': 'Vmax'
}

# List comprehension for the folder structure code
[os.makedirs(val) for key, val in config.items() if not os.path.exists(val)]

## 1. Test dilation of land mask on category 4 data

In [ ]:
# Set directory to save results
save_dir = f"{config['out_vmax']}/hm==23_dm!=0"
os.makedirs(save_dir, exist_ok=True)
use_cat0 = False

# Retrieve nc files only of category 4
file_list_cat4 = glob(f"{config['in_nc']}/category4/*.nc")
file_list_cat5 = [os.path.basename(x) for x in glob(f"{config['in_nc']}/category5/*.nc")]
list_cat4 = [file for file in file_list_cat4 if os.path.basename(file) not in file_list_cat5]
print(len(list_cat4))

#file_list_cat1 = glob(f"{config['in_nc']}/category1/*.nc")
#file_list_cat2 = [os.path.basename(x) for x in glob(f"{config['in_nc']}/category2/*.nc")]
#list_cat1 = [file for file in file_list_cat1 if os.path.basename(file) not in file_list_cat2]

values = []
for single_file in list_cat4:
    # Read .nc file
    full_info_image = netCDF4.Dataset(single_file, mode='r') 

    # Get wind_speed feature from .nc product
    feature_wind = full_info_image.variables["wind_speed"][:]
    feature_wind = feature_wind[0]

    # Plot feature
    plt.imshow(feature_wind)
    plt.title("Original wind feature")
    plt.colorbar()
    plt.show()
    print("First Vmax value:", np.max(feature_wind))
    ind = np.unravel_index(np.argmax(feature_wind, axis=None), feature_wind.shape)
    print("Pixel for the Vmax found:", ind)

    # Get mask from .nc product
    mask = full_info_image.variables["mask_flag"][:]
    mask = mask[0]
    mask[mask != 0] = 1
    #np.int8(mask != 0)

    # Dilate mask
    kernel = np.ones((11, 11), np.int8)
    dilation = cv2.dilate(mask,kernel,iterations = 1)
    
    #plt.imshow(dilation)
    #plt.colorbar()
    #plt.show()    
    
    # Apply dilated mask to feature
    feature_wind[dilation != 0] = 0

    plt.imshow(feature_wind)
    plt.title("Wind feature with land mask")
    plt.colorbar()
    plt.show()
    print("Intermediate Vmax value:", np.max(feature_wind))

    # Get heterogeneity mask from .nc product
    heterogeneity_mask = full_info_image.variables["heterogeneity_mask"][:]
    heterogeneity_mask = heterogeneity_mask[0]

    plt.imshow(heterogeneity_mask)
    plt.title("heterogeneity mask")
    plt.colorbar()
    plt.show()
    print("heterogeneity mask unique values:", np.unique(heterogeneity_mask))

    # Test feature with both masks
    feature_wind[heterogeneity_mask == 3] = 0
    feature_wind[heterogeneity_mask == 2] = 0
    plt.imshow(feature_wind)
    plt.title("Wind feature with both masks")
    plt.colorbar()
    plt.show()

    ind = np.unravel_index(np.argmax(feature_wind, axis=None), feature_wind.shape)
    ws_Vmax = np.max(feature_wind)
    print(os.path.basename(single_file))
    print("Final Vmax value:", ws_Vmax)
    print("Pixel for the Vmax found:", ind)

    values.append(ws_Vmax)
    #plt.imshow(feature_wind)
    #plt.title("Final image")
    #plt.colorbar()
    #plt.show()
    print("---------------------------------------")
print("Values:", values)

In [ ]:
plt.hist(values)
plt.title("Category 4 Vmax distribution")
plt.savefig(f"{save_dir}/Category4_Vmax_dist.png", bbox_inches='tight')

## 2. Create dataframe with all Vmax values for all categories

In [ ]:
# Load generic best tracks CSV file
TC_dataset = pd.read_csv("best_track/ibtracs.since1980.list.v04r00.csv", header=0)

# Load dataframe connecting .nc filename to TC Name and USA_ATCF_ID
nc_ID = pd.read_csv(f"{config['in_nc']}/cyclobs_tc_dataframe.csv", header=0)

# Gather all .nc files from all categories
file_list = glob(f"{config['in_nc']}/*/*.nc")

files = []
ids = []
cyclob_Vmax_list = []
ws_Vmax_list = []
best_track_Vmax_list = []
tmax_list = []
ws_Vmax_masked_list = []

cnt=0
for single_file in file_list:
    nc_filename = os.path.basename(single_file)
    df = nc_ID[nc_ID.data == nc_filename]

    # cyclob_Vmax is the Vmax value in the request url dataframe from Cyclobs API
    cyclob_Vmax = df['vmax (m/s)'].values
    id = df.sid.values

    ##############################
    try:
        # Read the .nc product
        full_info_image = netCDF4.Dataset(single_file, mode='r') 

        # Try to read "nrcs_detrend_cross" feature (may give problems!)
        feature_cross = full_info_image.variables["nrcs_detrend_cross"][:]

        tmax_units = datetime.strptime(full_info_image.measurementDate, '%Y-%m-%dT%H:%M:%SZ')
        tmax = tmax_units.strftime('%Y-%m-%d %H_%M_%S')

        # Extract wind speed info from .nc product
        feature_wind = full_info_image.variables["wind_speed"][:]
        feature_wind = feature_wind[0]

        # ws_Vmax is the maximum wind speed of the product
        ws_Vmax = np.max(feature_wind)
        #print(ws_Vmax)

        # Extract mask from .nc product
        mask = full_info_image.variables["mask_flag"][:]
        mask = mask[0]
        mask[mask != 0] = 1
        
        # Dilate mask
        kernel = np.ones((11, 11), np.int8)
        mask_dilated = cv2.dilate(mask, kernel, iterations = 1)

        # Extract heterogeneity mask from .nc product
        heterogeneity_mask = full_info_image.variables["heterogeneity_mask"][:]
        heterogeneity_mask = heterogeneity_mask[0]
        #print(np.unique(heterogeneity_mask))

        # Set all mask pixels different than 0 to 0
        feature_wind[mask_dilated != 0] = 0
        #feature_wind[heterogeneity_mask != 0] = 0
        feature_wind[heterogeneity_mask == 3] = 0
        feature_wind[heterogeneity_mask == 2] = 0

        # ws_Vmax_masked is the Vmax of the product after applying the masks
        ws_Vmax_masked = np.max(feature_wind)
        if ws_Vmax != ws_Vmax_masked:
            cnt +=1
        full_info_image.close()
    
    except KeyError as err:
        # Creating KeyError instance for book keeping
        print("Error:", err)
        full_info_image.close()
        continue
    ##############################

    # Get rows in the TC best track that are taken in the same day of the TC
    day = tmax[:10]
    img_info = TC_dataset.loc[TC_dataset.ISO_TIME.str.contains(day)]
    img_info = img_info.loc[img_info.USA_ATCF_ID == id[0].upper()]

    for index, row in img_info.iterrows():
        img_info['ISO_TIME'][index] = datetime.strptime(img_info['ISO_TIME'][index], '%Y-%m-%d %H:%M:%S')
    img_info['USA_WIND'] = img_info['USA_WIND'].astype(float)
    aux = img_info[['ISO_TIME', 'USA_WIND']]

    df2 = pd.DataFrame([[tmax_units, np.nan]], columns=['ISO_TIME', 'USA_WIND'])
    #print(df2)
    aux = aux.append(df2, ignore_index=True)
    aux = aux.sort_values(by='ISO_TIME')
    #print(aux)

    interp = aux.interpolate()
    #print(interp)
    best_track_Vmax = interp.USA_WIND.loc[interp.ISO_TIME == tmax_units].values
    #print(best_track_Vmax)

    files.append(single_file)
    ids.append(id[0])
    cyclob_Vmax_list.append(cyclob_Vmax[0])
    best_track_Vmax_list.append(knots_to_m_sec(best_track_Vmax[0]))
    ws_Vmax_list.append(ws_Vmax)
    ws_Vmax_masked_list.append(ws_Vmax_masked)
    tmax_list.append(tmax)

data = {"nc": files, "sid": ids, "tmax": tmax_list, "Vmax_ws": ws_Vmax_list, 
        "Vmax_ws_masked": ws_Vmax_masked_list, "Vmax_cyclob": cyclob_Vmax_list, 
        "Vmax_best_track": best_track_Vmax_list}
data_df = pd.DataFrame(data)

print("Number of Vmax values that changed by applying the land mask and the heterogeneity mask:", cnt)

# Save relevant information in a csv file
data_df.to_csv(f"{save_dir}/Vmax_info_masked_both.csv", index=False)

### 2.1. Drop duplicated rows and timestamps not present in our dataset

In [ ]:
#####################################################################################
# NOTE: 
# What reduces the dataset to a size of 227 is dropping duplicates and excluding .nc
# products that give an error while trying to read the nrcs_detrend_cross feature
#####################################################################################

dir = 'VV_VH_WS'

dataset = pd.read_csv(f"{save_dir}/Vmax_info_masked_both.csv")

# Check indexes with duplicated tmax and keep the last one
dataset = dataset.drop_duplicates(subset=["tmax"], keep = 'last')

# Eliminate rows with timestamps not present in our dataset
file_list = glob(f"{dir}/*/*.png")

times = np.array([item.split(os.sep)[-1][:-4] for item in file_list])
df_tmax = dataset.tmax.values

# This diff happened because I wasn't using try except nrcs_detrend_cross exception
diff = list(list(set(times)-set(df_tmax)) + list(set(df_tmax)-set(times)))
if len(diff) > 0:
    print(diff)
    idxs = []
    for item in diff:
        df = dataset.loc[dataset.tmax == item]

        for idx, row in df.iterrows():
            idxs.append(idx)
    dataset = dataset.drop(idxs)

# Create category column
dataset["category"] = dataset["nc"].str.split(os.sep, expand=True)[1]

if use_cat0:
    # Load full_dataset.csv to get info on labels
    df = pd.read_csv(f"{dir}/csv/full_dataset.csv")
    df['tmax'] = df['image'].str.split(os.sep).str[-1].str[:-4]
    #print(df)
    dataset = dataset.merge(df[['tmax', 'label']])
    
    # Drop products where there is no eye
    #dataset = dataset[dataset.label != 0]
    # Instead of dropping no-eye images, we can create "category0"
    dataset["category"] = np.where((dataset.label == 0), 'category0', dataset["category"])

dataset.reset_index(inplace=True, drop=True)
dataset.to_csv(f"{save_dir}/Vmax_info_masked_both_filtered.csv", index=False)

print("cat0:", len(dataset[dataset.category == "category0"]))
print("cat1:", len(dataset[dataset.category == "category1"]))
print("cat2:", len(dataset[dataset.category == "category2"]))
print("cat3:", len(dataset[dataset.category == "category3"]))
print("cat4:", len(dataset[dataset.category == "category4"]))
print("cat5:", len(dataset[dataset.category == "category5"]))
dataset

### 2.2. Plot distribution of Vmax across categories for different variables.

In [ ]:
hist_dir = f"{save_dir}/histograms"
os.makedirs(hist_dir, exist_ok=True)
save_plots = True
n_bins = 50

dataset = pd.read_csv(f"{save_dir}/Vmax_info_masked_both_filtered.csv")

#######################
###   Vmax_cyclob   ###
#######################
fig, ax = plt.subplots(figsize =(11,8))
rate = 0.2
for cat in range(1, 6):
    category = 'category'+str(cat)
    feature = dataset.where(dataset.category == category).loc[:, "Vmax_cyclob"]
    feature.hist(ax=ax, label=category)
ax.set(title='Vmax cyclob', ylabel='Count', xlabel='Vmax (m/s)')
ax.legend(fontsize = 15)
ax.grid(axis='y')
ax.set_facecolor('#d8dcd6')
plt.show()
if save_plots:
    fig.savefig(f"{hist_dir}/Vmax_cyclob.png", bbox_inches='tight')


##########################
###   Vmax_ws_masked   ###
##########################
fig, (ax1, ax2) = plt.subplots(1, 2, figsize =(19,8))

for cat in range(1, 6):
    category = 'category'+str(cat)
    feature = dataset.where(dataset.category == category).loc[:, "Vmax_ws_masked"]
    feature.hist(ax=ax1, label=category, alpha = rate*cat, bins = n_bins)
ax1.set(title='Vmax masked unstacked', ylabel='Count', xlabel='Vmax (m/s)')
ax1.legend(fontsize = 15)
ax1.grid(axis='y')

cats = ['category1', 'category2', 'category3', 'category4', 'category5']
cat1 = dataset.where(dataset.category == 'category1').loc[:, "Vmax_ws_masked"]
cat2 = dataset.where(dataset.category == 'category2').loc[:, "Vmax_ws_masked"]
cat3 = dataset.where(dataset.category == 'category3').loc[:, "Vmax_ws_masked"]
cat4 = dataset.where(dataset.category == 'category4').loc[:, "Vmax_ws_masked"]
cat5 = dataset.where(dataset.category == 'category5').loc[:, "Vmax_ws_masked"]

ax2.hist([cat1, cat2, cat3, cat4, cat5], stacked=True, label = cats, bins = n_bins)
ax2.set(title='Vmax masked stacked', ylabel='Count', xlabel='Vmax (m/s)')
ax2.legend(fontsize = 15)
ax2.grid(axis='x')
ax2.set_facecolor('#d8dcd6')
plt.show()
if save_plots:
    fig.savefig(f"{hist_dir}/Vmax_ws_masked.png", bbox_inches='tight')


###################
###   Vmax_ws   ###
###################
fig, (ax1, ax2) = plt.subplots(1, 2, figsize =(19,8))

for cat in range(1, 6):
    category = 'category'+str(cat)
    feature = dataset.where(dataset.category == category).loc[:, "Vmax_ws"]
    feature.hist(ax=ax1, label=category, alpha = rate*cat, bins = n_bins)
ax1.set(title='Vmax unmasked unstacked', ylabel='Count', xlabel='Vmax (m/s)')
ax1.legend(fontsize = 15)
ax1.grid(axis='y')
#ax.set_facecolor('#d8dcd6')

cats = ['category1', 'category2', 'category3', 'category4', 'category5']
cat1 = dataset.where(dataset.category == 'category1').loc[:, "Vmax_ws"]
cat2 = dataset.where(dataset.category == 'category2').loc[:, "Vmax_ws"]
cat3 = dataset.where(dataset.category == 'category3').loc[:, "Vmax_ws"]
cat4 = dataset.where(dataset.category == 'category4').loc[:, "Vmax_ws"]
cat5 = dataset.where(dataset.category == 'category5').loc[:, "Vmax_ws"]

ax2.hist([cat1, cat2, cat3, cat4, cat5], stacked=True, label = cats, bins = n_bins)
ax2.set(title='Vmax unmasked stacked', ylabel='Count', xlabel='Vmax (m/s)')
ax2.legend(fontsize = 15)
ax2.grid(axis='x')
ax2.set_facecolor('#d8dcd6')
plt.show()
if save_plots:
    fig.savefig(f"{hist_dir}/Vmax_ws.png", bbox_inches='tight')

## 3. Train Regression model

In [ ]:
# Importing the dataset
results_dir = f"{save_dir}/regression_results"
os.makedirs(results_dir, exist_ok= True)
save_plots = True

dataset = pd.read_csv(f"{save_dir}/Vmax_info_masked_both_filtered.csv")

feature = "Vmax_ws_masked"
X = dataset.loc[:, feature].values
y = dataset.loc[:, "category"].str.extract('(\d+)').astype(int).values.flatten()
#X = X/X.max()
#print(X)

# Split the dataset into the Training set and Test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1, stratify=y)
#print(X_test, y_test)

# One-hot encode outputs
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

# Count number of classes
nb_classes = y_test.shape[1]
print("Number of classes:", nb_classes)

X_train = X_train.reshape(-1, 1)
X_test = X_test.reshape(-1, 1)

# Feature scaling
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)
#print(X_train, X_test)

# Create model
model = Sequential()
model.add(Dense(32, activation = 'relu', input_dim = 1))    # adding the input layer and the first hidden layer
model.add(Dense(units = 32, activation = 'relu'))   # second hidden layer
model.add(Dense(units = 32, activation = 'relu'))   # third hidden layer 
model.add(Dense(nb_classes, activation='softmax'))   # output layer
#model.add(Dense(units = 1))

# Compile model
model.compile(optimizer = 'adam', 
              loss = 'categorical_crossentropy', #loss = 'mean_squared_error'
              metrics = ['accuracy'])

# Fit model to data
callback = [
    ModelCheckpoint(f"{results_dir}/{feature}_best_model.h5", verbose=1, save_best_only=True)
]

history = model.fit(
    X_train, y_train,
    validation_split = 0.2,
    batch_size = 10,
    epochs = 50,
    callbacks = callback
)

# Load best model
time.sleep(5) # guarantees enough time for weights to be saved and loaded afterwards, otherwise gives concurrency problems
print("Loaded best weights of the training")
model.load_weights(f"{results_dir}/{feature}_best_model.h5")

# Make predictions
y_pred = model.predict(X_test)
#print(y_pred)
predictions = [np.argmax(t) for t in y_pred]
#print(predictions)
y_test_non_category = [np.argmax(t) for t in y_test]

# Plot confusion matrix
conf_mat = confusion_matrix(y_test_non_category, predictions, labels = [0,1,2,3,4,5])
disp = ConfusionMatrixDisplay(confusion_matrix = conf_mat, display_labels = [0,1,2,3,4,5])
conf_mat_display = disp.plot()
if save_plots:
    plt.savefig(f"{results_dir}/{feature}_confusion_matrix.jpg", bbox_inches='tight')

# Plot train/validation losses
fig, (ax1, ax2) = plt.subplots(nrows=2, ncols=1, figsize=(10,10))
ax1.plot(history.history['accuracy'], label='Training Accuracy')
ax1.plot(history.history['val_accuracy'], label='Validation Accuracy')
ax1.grid(True)
ax1.legend(loc='lower right')
ax1.set(ylabel="Accuracy",
        title='Training and Validation Accuracy')

ax2.plot(history.history['loss'], label='Training Loss')
ax2.plot(history.history['val_loss'], label='Validation Loss')
ax2.grid(True)
ax2.legend(loc='upper right')
ax2.set(xlabel='Epoch',
        ylabel='Categorical Cross Entropy',
        title='Training and Validation Loss')
plt.show()
if save_plots:
    fig.savefig(f"{results_dir}/{feature}_learning_curves.jpg", bbox_inches='tight')

In [ ]:
# Plot predictions vs labels
fig, ax = plt.subplots(figsize=(10, 10))
ax.plot(y_test_non_category, '.', color = 'red', label = 'Real data')
ax.plot(predictions, '.', color = 'blue', label = 'Predicted data')
ax.set(title = 'Prediction')
ax.legend()
plt.show()

### 3.1. Test different regression model structures

In [ ]:
# model = tf.keras.Sequential([
#   tf.keras.layers.Dense(10, activation=tf.nn.relu, input_shape=(1,)),  # input shape required
#   tf.keras.layers.Dense(10, activation=tf.nn.relu),
#   tf.keras.layers.Dense(3)
# ])


# model = Sequential()
# model.add(Dense(10, activation='relu', input_dim=1))
# model.add(Dense(10, activation='relu'))
# model.add(Dense(5, activation='softmax'))